# Day 3: Disney Parks Guest Spending Behavior

You are a data analyst working with the Disney Parks revenue team to understand nuanced guest spending patterns across different park experiences. The team wants to develop a comprehensive view of visitor purchasing behaviors. Your goal is to uncover meaningful insights that can drive personalized marketing strategies.

In [ ]:
import pandas as pd
import numpy as np

fct_guest_spending_data = [
  {
    "guest_id": 1,
    "visit_date": "2024-07-05",
    "amount_spent": 50,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-07-06",
    "amount_spent": 30,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-07-10",
    "amount_spent": 20.5,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 4,
    "visit_date": "2024-07-12",
    "amount_spent": 40,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-07-15",
    "amount_spent": 35,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 5,
    "visit_date": "2024-07-20",
    "amount_spent": 60,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 6,
    "visit_date": "2024-07-25",
    "amount_spent": 25,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-08-03",
    "amount_spent": 55,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-08-15",
    "amount_spent": 45,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-08-05",
    "amount_spent": 22,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-08-20",
    "amount_spent": 38,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 7,
    "visit_date": "2024-08-10",
    "amount_spent": 15,
    "park_experience_type": "Character Meet"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-08-25",
    "amount_spent": 28,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-08-27",
    "amount_spent": 32,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-09-02",
    "amount_spent": 65,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 8,
    "visit_date": "2024-09-05",
    "amount_spent": 50,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 9,
    "visit_date": "2024-09-15",
    "amount_spent": 40,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 10,
    "visit_date": "2024-09-20",
    "amount_spent": 70,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-09-25",
    "amount_spent": 35,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 8,
    "visit_date": "2024-09-28",
    "amount_spent": 10,
    "park_experience_type": "Character Meet"
  }
]
fct_guest_spending = pd.DataFrame(fct_guest_spending_data)


## Question 1

What is the average spending per guest per visit for each park experience type during July 2024? Ensure that park experience types with no recorded transactions are shown with an average spending of 0.0. This analysis helps establish baseline spending differences essential for later segmentation.

In [ ]:
fct_guest_spending['visit_date'] = pd.to_datetime(fct_guest_spending['visit_date'])

july_spending = fct_guest_spending[
    (fct_guest_spending['visit_date'].dt.month == 7) &
    (fct_guest_spending['visit_date'].dt.year == 2024)
]

avg_spending = july_spending.groupby(july_spending['park_experience_type'])['amount_spent'].mean().reset_index()
avg_spending.rename(columns={'amount_spent' : 'avg_spending'}, inplace=True)

all_types = fct_guest_spending['park_experience_type'].unique()
avg_spending = avg_spending.set_index('park_experience_type').reindex(all_types, fill_value=0.0).reset_index()
avg_spending['avg_spending'] = avg_spending['avg_spending'].round(2)

print("Average spending per guest for each park experience type (July 2024): ", avg_spending.to_string(index=False))

## Question 2

For guests who visited our parks more than once in August 2024, what is the difference in spending between their first and their last visit? This investigation, using sequential analysis, will reveal any shifts in guest spending behavior over multiple visits.

In [ ]:
fct_guest_spending['visit_date'] = pd.to_datetime(fct_guest_spending['visit_date'])

august_spending = fct_guest_spending[
    (fct_guest_spending['visit_date'].dt.month == 8) &
    (fct_guest_spending['visit_date'].dt.year == 2024)
]

august_spending = august_spending.sort_values(['guest_id', 'visit_date'])

first_visit = august_spending.groupby('guest_id').first().reset_index()
last_visit = august_spending.groupby('guest_id').last().reset_index()

merge_visits = pd.merge(
    first_visit[['guest_id', 'amount_spent']],
    last_visit[['guest_id', 'amount_spent']],
    on = 'guest_id',
    suffixes = ('_first', '_last')
)
merge_visits['spending_diff'] = merge_visits['amount_spent_last']-merge_visits['amount_spent_first']
visit_count = august_spending.groupby('guest_id').size().reset_index(name = 'visit_count')
multi_visitors = visit_count[visit_count['visit_count'] > 1]

final_result = merge_visits[merge_visits['guest_id'].isin(multi_visitors['guest_id'])]

print("Difference in spending between first and last visit for multi-visit guests in Aug 2024:")
print(final_result[['guest_id', 'amount_spent_first', 'amount_spent_last', 'spending_diff']])

## Question 3

In September 2024, how can guests be categorized into distinct spending segments such as Low, Medium, and High based on their total spending? Use the following thresholds for categorization: 
-Low: Includes values from $0 up to, but not including, $50.
-Medium: Includes values from $50 up to, but not including, $100.
-High: Includes values from $100 and above. 
Exclude guests who did not make any purchases in the period.

In [ ]:
fct_guest_spending['visit_date'] = pd.to_datetime(fct_guest_spending['visit_date'])

sept_spending = fct_guest_spending[
    (fct_guest_spending['visit_date'].dt.month == 9) &
    (fct_guest_spending['visit_date'].dt.year == 2024)
]
sept_spending = sept_spending.groupby('guest_id')['amount_spent'].sum().reset_index()
sept_spending = sept_spending[sept_spending['amount_spent'] > 0]

bins=[0, 50, 100, float('inf')]
labels=['Low', 'Medium', 'High']

sept_spending['spending_segment'] = pd.cut(
    sept_spending['amount_spent'],
    bins=bins,
    labels=labels,
    right=False,
    include_lowest=True
)
print('September 2024 guest spending segments:')
print(sept_spending[['guest_id', 'amount_spent', 'spending_segment']].to_string(index=False))

segment_counts = sept_spending['spending_segment'].value_counts().reindex(labels, fill_value=0)
print('\nCount by segment:')
print(segment_counts.to_string())

Made with ❤️ by [Interview Master](https://www.interviewmaster.ai)